In [14]:
import os
import requests
from datetime import datetime
from google.cloud import storage
from google.oauth2 import service_account

# Configurações
base = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
year = 2010
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
limit = 50000

gcs_bucket_name = "projeto_nyc"

# Credenciais
credentials = service_account.Credentials.from_service_account_file(
    r'C:\Users\izabela.pereira\Documents\projetos\projeto_nyc\service_account.json'
)
client = storage.Client(credentials=credentials)
bucket = client.bucket(gcs_bucket_name)

for month in months:
    print(f'\n🔄 Processando {year}-{month:02d}')
    offset = 0
    total_rows = 0

    start_date = datetime(year, month, 1)

    if month == 12:
        next_month = datetime(year + 1, 1, 1)
    else:
        next_month = datetime(year, month + 1, 1)

    where = (
        f'created_date >= "{start_date.isoformat()}" AND '
        f'created_date < "{next_month.isoformat()}"'
    )

    while True:
        params = {
            "$where": where,
            "$limit": limit,
            "$offset": offset
        }

        resp = requests.get(base, params=params, timeout=60)
        resp.raise_for_status()
        content = resp.text

        lines = content.strip().split('\n')
        if len(lines) <= 1:
            break

        num_rows = len(lines) - 1

        output_filename = f'nyc_311_{year}_{month:02d}_offset_{offset}.csv'
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(content)

        gcs_blob_name = (
            f'landing/nyc_311/'
            f'nyc_311_{year}_{month:02d}_offset_{offset}.csv'
        )

        bucket.blob(gcs_blob_name).upload_from_filename(output_filename)

        print(f'✔ Enviado {num_rows} registros | offset {offset}')

        os.remove(output_filename)

        total_rows += num_rows
        offset += limit

    print(f'✅ {year}-{month:02d} finalizado com {total_rows} registros')



🔄 Processando 2010-01
✔ Enviado 135808 registros | offset 0
✔ Enviado 135938 registros | offset 50000
✔ Enviado 136052 registros | offset 100000
✔ Enviado 104017 registros | offset 150000
✅ 2010-01 finalizado com 511815 registros

🔄 Processando 2010-02
✔ Enviado 136524 registros | offset 0
✔ Enviado 135586 registros | offset 50000
✔ Enviado 135198 registros | offset 100000
✔ Enviado 37393 registros | offset 150000
✅ 2010-02 finalizado com 444701 registros

🔄 Processando 2010-03
✔ Enviado 135114 registros | offset 0
✔ Enviado 134974 registros | offset 50000
✔ Enviado 135210 registros | offset 100000
✔ Enviado 133224 registros | offset 150000
✔ Enviado 13729 registros | offset 200000
✅ 2010-03 finalizado com 552251 registros

🔄 Processando 2010-04
✔ Enviado 134302 registros | offset 0
✔ Enviado 133622 registros | offset 50000
✔ Enviado 134068 registros | offset 100000
✔ Enviado 51751 registros | offset 150000
✅ 2010-04 finalizado com 453743 registros

🔄 Processando 2010-05
✔ Enviado 136